In [0]:
#This Code predict Label for SVF Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_20102019_Single.csv')
data = data.sample(frac=1)

y = list(data['Label']) 
x = list(data['Title_Description']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [12]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=60, batch_size=32)
    model.save_weights('./Roslyn_TD_Label_Predict_Single.h5')

Epoch 1/60
715/715 [==============================] - 61s 85ms/step - loss: 2.2267 - acc: 0.5790
Epoch 2/60
715/715 [==============================] - 58s 81ms/step - loss: 1.2610 - acc: 0.6517
Epoch 3/60
715/715 [==============================] - 58s 80ms/step - loss: 1.0857 - acc: 0.6881
Epoch 4/60
715/715 [==============================] - 58s 80ms/step - loss: 1.0066 - acc: 0.6979
Epoch 5/60
715/715 [==============================] - 57s 80ms/step - loss: 0.9551 - acc: 0.7119
Epoch 6/60
715/715 [==============================] - 61s 85ms/step - loss: 0.9158 - acc: 0.7217
Epoch 7/60
715/715 [==============================] - 57s 80ms/step - loss: 0.8928 - acc: 0.7231
Epoch 8/60
715/715 [==============================] - 57s 80ms/step - loss: 0.8644 - acc: 0.7315
Epoch 9/60
715/715 [==============================] - 57s 80ms/step - loss: 0.8487 - acc: 0.7357
Epoch 10/60
715/715 [==============================] - 57s 80ms/step - loss: 0.8350 - acc: 0.7427
Epoch 11/60
715/715 [========

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./Roslyn_TD_Label_Predict_Single.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [16]:
y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))



                             precision    recall  f1-score   support

              4 - In Review       0.00      0.00      0.00         2
          Approved to merge       0.00      0.00      0.00         1
             Area-Analyzers       0.33      0.20      0.25         5
             Area-Compilers       0.58      0.67      0.62        48
              Area-External       0.00      0.00      0.00         2
                   Area-IDE       0.70      0.75      0.72        75
        Area-Infrastructure       0.84      0.91      0.88        35
           Area-Interactive       0.00      0.00      0.00         5
       Area-Language Design       0.00      0.00      0.00         1
                  Community       0.00      0.00      0.00         1
PR For Personal Review Only       0.00      0.00      0.00         2
       Resolution-Duplicate       0.00      0.00      0.00         2
                        nan       0.00      0.00      0.00         0

                   accuracy     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# New Section